### 8.2 Tiền xử lý dữ liệu văn bản
Dữ liệu văn bản là một ví dụ điển hình của dữ liệu chuỗi. Một bài báo có thể coi là một chuỗi các từ hoặc một chuỗi các ký tự. 
- Quá trình tiền xử lý dữ liệu thường bao gồm bốn bước sau:
    1. Nạp dữ liệu văn bản ở dạng chuỗi ký tự vào bộ nhớ.
    2. Chia chuỗi thành các token trong đó một token có thể là một từ hoặc một ký tự.
    3. Xây dựng một bộ từ vựng cho các token để ánh xạ chúng thành các chỉ số (index)
    4. Ánh xạ tất cả các token trong dữ liệu văn bản thành các chỉ số để dễ dàng đưa vào mô hình.

In [1]:
import collections
import random
import re
import torch
from d2l import torch as d2l

Để bắt đầu ta sẽ nạp dữ liệu văn bản từ cuốn sách Time Machine của tác giả H.G Wells. Đây là một kho dữ liệu khá nhỏ chỉ hơn 30000 từ nhưng nó đủ tốt cho mục đích minh họa. Nhiều bộ dữ liệu trên thực tế chứa hàng tỷ từ. 
- Đoạn code sau đây sẽ tải về bộ dữ liệu TimeMachine:

In [2]:
class TimeMachine(d2l.DataModule): #@save
    """The Time Machine dataset."""
    def _download(self):
        fname = d2l.download(d2l.DATA_URL + 'timemachine.txt', self.root,
                             '090b5e7e70c295757f55df93cb0a180b9691891a')
        with open(fname) as f:
            return f.readlines()

data = TimeMachine()
raw_text = data._download()
raw_text[:10]

['The Time Machine, by H. G. Wells [1898]\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'I\n',
 '\n',
 '\n',
 'The Time Traveller (for so it will be convenient to speak of him)\n',
 'was expounding a recondite matter to us. His grey eyes shone and\n']

Để đơn giản, ta sẽ bỏ qua dấu câu và viết hoa trước khi xử lý văn bản.

In [3]:
@d2l.add_to_class(TimeMachine)
def _preprocess(self, lines):
    return [re.sub('[^A-Za-z]+', ' ', line).lower().strip() for line in lines]

lines = data._preprocess(raw_text)
lines[:10]

['the time machine by h g wells',
 '',
 '',
 '',
 '',
 'i',
 '',
 '',
 'the time traveller for so it will be convenient to speak of him',
 'was expounding a recondite matter to us his grey eyes shone and']

#### Token hóa
- Với mỗi câu, chúng ta sẽ chia nó thành một danh sách các token.
- Một token là một điểm dữ liệu mà mô hình sẽ huấn luyện và đưa ra dự đoán từ nó. 
- Hàm dưới đây làm nhiệm vụ tách một câu thành các từ hoặc các ký tự và trả về một danh sách các chuỗi đã được phân tách.

In [4]:
@d2l.add_to_class(TimeMachine)
def _tokenize(self, lines, token = 'word'):
    if token == 'word':
        return [line.split(' ') for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    

tokens = data._tokenize(lines)
tokens[0:3]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'], [''], ['']]

#### 3. Bộ từ vựng
- Token kiểu chuỗi không phải kiểu dữ liệu tiện lợi được sử dụng bởi các mô hình, thay vào đó chúng thường nhận dữ liệu đầu vào dưới dạng số.
- Ta xây dựng một bộ từ điển, thường được gọi là __bộ từ vựng (vocabulary)__, để ánh xạ chuỗi token thành các chỉ số bắt đầu từ 0.
- Để làm điều này, 
    1. đầu tiên ta lấy các token xuất hiện (không lặp lại) trong toàn bộ tài liệu, thường được gọi là __kho ngữ liệu (corpus)__.
    2. Gán một giá trị số cho mỗi token dựa trên tần suất xuất hiện của chúng, các token có tần suất xuất hiện rất ít thường bị loại bỏ để giảm độ phức tạp. 
    3. Mỗi token không xuất hiện trong kho ngữ liệu hoặc đã bị loại bỏ thường được ánh xạ vào một token vô danh đặc biệt.

In [5]:
def count_corpus(sentences):
    tokens = []
    for line in sentences:
        for token in line:
            tokens.append(token)
    return collections.Counter(tokens)

In [6]:
counter = count_corpus(tokens)
token_freqs = sorted(counter.items(), key = lambda x: x[0])
print("Sort by key: ", token_freqs)
token_freqs.sort(key = lambda x: x[1], reverse = True)
print("Sort by value (max to min): ", token_freqs)

Sort by key:  [('', 373), ('a', 816), ('abandon', 1), ('abandoned', 1), ('able', 3), ('abnormally', 1), ('abominable', 2), ('abominations', 1), ('about', 70), ('above', 23), ('abruptly', 4), ('absence', 4), ('absent', 1), ('absolute', 7), ('absolutely', 11), ('absorbed', 1), ('abstain', 1), ('abstract', 1), ('abstractions', 1), ('absurd', 1), ('abundance', 1), ('abundant', 8), ('abysmal', 1), ('abysses', 1), ('acacias', 1), ('academic', 1), ('accelerate', 1), ('accept', 1), ('accepted', 5), ('accession', 1), ('accident', 1), ('accompany', 1), ('accordingly', 2), ('account', 7), ('accumulate', 1), ('accustomed', 2), ('ached', 1), ('acquaintance', 1), ('across', 22), ('action', 1), ('active', 1), ('activity', 1), ('actual', 3), ('actually', 1), ('acutely', 1), ('adaptations', 1), ('adapted', 3), ('addition', 2), ('addressed', 1), ('adjacent', 1), ('adjoining', 1), ('adjustment', 2), ('admired', 1), ('admit', 4), ('admitted', 1), ('adorn', 1), ('adroitly', 1), ('advance', 1), ('advanced',

In [8]:
class Vocab:
    def __init__(self, tokens, min_freq = 0, reserved_tokens = None):
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key = lambda x: x[0])
        self.token_freqs.sort(key = lambda x: x[1], reverse = True)

        self.unk = 0
        uniq_tokens = ['<unk>'] + reserved_tokens

        # Lấy ra các token thỏa mãn điều kiện tần suất xuất hiện tối thiểu
        # Và không ở trong danh sách uniq_tokens trước đó
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens
        ]

        self.idx_to_token, self.token_to_idx = [], dict()
    
        # Tạo danh sách và dict để chuyển đổi từ số sang token và từ token thành số
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    # Trả về độ lớn của vocab
    def __len__(self):
        return len(self.idx_to_token)
    
    # Trả về các token của vocab
    # Sử dụng đệ quy để giải quyết trường hợp các token ở trong các list hoặc tuple lồng nhau
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            # Nếu tìm thấy thì trả về chỉ số của token trong vocab, 
            # Nếu không thì trả về self.unk (0)
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    # Chuyển một danh sách/ tuple các chỉ số thành token
    def to_tokens(self, indices):
        # Nếu indices không phải là list, tuple mà là scalar
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(sentences):
    tokens = [tk for line in sentences for tk in line]
    return collections.Counter(tokens)
        

In [12]:
vocab = Vocab(tokens)
print(
    list(vocab.token_to_idx.items())[0:10]
)

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [18]:
tokens = [token for token in tokens if token != ['']]
tokens[3]

['was',
 'expounding',
 'a',
 'recondite',
 'matter',
 'to',
 'us',
 'his',
 'grey',
 'eyes',
 'shone',
 'and']

In [19]:
print("Words: ", tokens[3])
print("Indices: ", vocab[tokens[3]])

Words:  ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
Indices:  [7, 1654, 5, 3864, 634, 6, 131, 26, 344, 127, 484, 3]


In [20]:
print("Vocab size: ", vocab.__len__())

Vocab size:  4581
